In [ ]:
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials

# 1. Define the scope
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets"
]

# 2. Authenticate using your Service Account key file
# Replace with the actual path to your JSON key file
SERVICE_ACCOUNT_FILE = '/Users/deepshah/Downloads/tiffinstash-key.json' 

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

# 3. Open the Google Sheet
# You can open by title, url, or key. Opening by key is most robust.
# Example URL: https://docs.google.com/spreadsheets/d/1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms/edit
SHEET_ID = '1yN7oUVMW_5R-JzEVwGQl26xnvUFay3aGWoudG5aMH48' 

try:
    # Open the spreadsheet
    sh = client.open_by_key(SHEET_ID)
    
    # Select the specific worksheet (tab) by index (0 is the first one) or name
    worksheet = sh.get_worksheet(3) 
    # OR: worksheet = sh.worksheet("Sheet1")

    # 4. Get all values and convert to DataFrame
    data = worksheet.get_all_records() # Returns a list of dictionaries
    df = pd.DataFrame(data)

    print("Data extracted successfully:")
    print(df.head())

except Exception as e:
    print(f"Error reading Google Sheet: {e}")



In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import text
from google.cloud.sql.connector import Connector, IPTypes
import os
from google.oauth2 import service_account



# # 1. Load your local CSV data
# file_path = os.path.expanduser('~/Downloads/historical-data-filtered.csv')
# df = pd.read_csv(file_path)

# 2. Cleanup dates as you did before
for col in df.columns:
    if df[col].dtype == 'object':
        try:
            df[col] = pd.to_datetime(df[col], dayfirst=True, format='mixed')
        except (ValueError, TypeError):
            continue

KEY_PATH = "/Users/deepshah/Downloads/tiffinstash-key.json"

credentials = service_account.Credentials.from_service_account_file(
    KEY_PATH
)

connector = Connector(credentials=credentials)

def getconn():
    conn = connector.connect(
        "pelagic-campus-484800-b3:us-central1:tiffinstash-master",
        "pg8000",
        user="postgres",
        password="tiffinstash2026",
        db="postgres",
        ip_type=IPTypes.PUBLIC
    )
    return conn

# 4. Create the SQLAlchemy Engine
engine = sqlalchemy.create_engine("postgresql+pg8000://", creator=getconn)

# 5. Import to the new table
# We use double quotes in the name because of the hyphen
table_name = 'historical-data'

try:
    # 'if_exists=replace' will create the table if it doesn't exist
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"Successfully imported {len(df)} rows into '{table_name}'")
    
    # Verification query
    with engine.connect() as conn:
        res = conn.execute(text(f'SELECT COUNT(*) FROM "{table_name}"'))
        print(f"Verified row count in DB: {res.fetchone()[0]}")

except Exception as e:
    print(f"Import failed: {e}")

# Always clean up the connector
# connector.close()

In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import text
from google.cloud.sql.connector import Connector, IPTypes
import os
from google.oauth2 import service_account


KEY_PATH = "/Users/deepshah/Downloads/tiffinstash-key.json"

credentials = service_account.Credentials.from_service_account_file(
    KEY_PATH
)

connector = Connector(credentials=credentials)

def getconn():
    conn = connector.connect(
        "pelagic-campus-484800-b3:us-central1:tiffinstash-master",
        "pg8000",
        user="postgres",
        password="tiffinstash2026",
        db="postgres",
        ip_type=IPTypes.PUBLIC
    )
    return conn

# 4. Create the SQLAlchemy Engine
engine = sqlalchemy.create_engine("postgresql+pg8000://", creator=getconn)

# 5. Import to the new table
# We use double quotes in the name because of the hyphen
table_name = 'historical-data'

with engine.connect() as conn:
    df = pd.read_sql(text(f'SELECT * FROM "{table_name}"'), conn)
    print(f"Verified row count in DB: {df.shape[0]}")
pd.set_option('display.max_columns', None)
df